# 1. Lectura y visualización de un dataset de segemntación


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!unzip "/content/drive/MyDrive/Cursos/object_detection/data/segmentation.zip"

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf

from sklearn.model_selection import train_test_split

In [ ]:
dataset_path = "./"
filenames = os.listdir(dataset_path)
filenames.sort()

image_list = []
mask_list = []

for filename in filenames:
  if (len(filename.split(".")) > 1 and filename.split(".")[1][:3] == "png"):
    if filename.split(".")[0][-2:] == "_L":
      mask_list.append(dataset_path + "/" + filename)
    else:
      image_list.append(dataset_path + "/" + filename)

In [ ]:
train_input_img_path, val_input_img_path, train_target_mask_paths, val_input_mask_paths = train_test_split(
    image_list,
    mask_list,
    test_size=0.2,
    random_state=42)

In [ ]:
def load_data(images_path, mask_path):
  samples = {"images": [], "masks":[]}

  for i in range(len(images_path)):
    img = plt.imread(images_path[i])
    mask = plt.imread(mask_path[i])
    img = cv2.resize(img, (256, 256))
    mask = cv2.resize(mask, (256, 256))

    samples["images"].append(img)
    samples["masks"].append(mask)
  
  samples = {
      "images": np.array(samples["images"]),
      "masks": np.array(samples["masks"])
  }

  return samples


In [ ]:
train_samples = load_data(train_input_img_path, train_target_mask_paths)
test_samples = load_data(val_input_img_path, val_input_mask_paths)

In [ ]:
plt.figure(figsize = (20,20))
plt.subplot(1,2,1)
plt.imshow(train_samples['images'][10])
plt.subplot(1,2,2)
plt.imshow(train_samples["masks"][10])

plt.show()

#2. Creación de la red U-net para segementación.

In [ ]:
def create_conv_block(input_tensor, num_filters):
  x = tf.keras.layers.Conv2D(
        filters = num_filters, 
        kernel_size = (3,3), 
        kernel_initializer = "he_normal",
        padding="same")(input_tensor)

  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation("relu")(x)


  x = tf.keras.layers.Conv2D(
        filters = num_filters, 
        kernel_size = (3,3), 
        kernel_initializer = "he_normal",
        padding="same")(x)
        
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation("relu")(x)

  return x

  


In [ ]:
def create_unet(input_shape, num_filters = 16, dropout = 0.1):

  # Encoder
  c1 = create_conv_block(input_shape, num_filters * 1)
  p1 = tf.keras.layers.MaxPooling2D((2,2))(c1)
  p1 = tf.keras.layers.Dropout(dropout)(p1)

  c2 = create_conv_block(p1, num_filters * 2)
  p2 = tf.keras.layers.MaxPooling2D((2,2))(c2)
  p2 = tf.keras.layers.Dropout(dropout)(p2)

  c3 = create_conv_block(p2, num_filters * 4)
  p3 = tf.keras.layers.MaxPooling2D((2,2))(c3)
  p3 = tf.keras.layers.Dropout(dropout)(p3)

  c4 = create_conv_block(p3, num_filters * 8)
  p4 = tf.keras.layers.MaxPooling2D((2,2))(c4)
  p4 = tf.keras.layers.Dropout(dropout)(p4)

  c5 = create_conv_block(p4, num_filters * 16)
  
  # Decoder

  u6 = tf.keras.layers.Convolution2DTranspose(num_filters*8, (3, 3), strides=(2, 2), padding='same')(c5)
  u6 = tf.keras.layers.concatenate([u6, c4])
  u6 = tf.keras.layers.Dropout(dropout)(u6)
  c6 = create_conv_block(u6, num_filters*8)


  u7 = tf.keras.layers.Convolution2DTranspose(num_filters*4, (3, 3), strides=(2, 2), padding='same')(c6)
  u7 = tf.keras.layers.concatenate([u7, c3])
  u7 = tf.keras.layers.Dropout(dropout)(u7)
  c7 = create_conv_block(u7, num_filters*4)

  u8 = tf.keras.layers.Convolution2DTranspose(num_filters*2, (3, 3), strides=(2, 2), padding='same')(c7)
  u8 = tf.keras.layers.concatenate([u8, c2])
  u8 = tf.keras.layers.Dropout(dropout)(u8)
  c8 = create_conv_block(u8, num_filters*2)

  u9 = tf.keras.layers.Convolution2DTranspose(num_filters*1, (3, 3), strides=(2, 2), padding='same')(c8)
  u9 = tf.keras.layers.concatenate([u9, c1])
  u9 = tf.keras.layers.Dropout(dropout)(u9)
  c9 = create_conv_block(u9, num_filters*1)

  output = tf.keras.layers.Conv2D(3, (1,1), activation="sigmoid")(c9)

  model = tf.keras.Model(inputs=[input_shape], outputs = [output])
  return model


In [ ]:
inputs = tf.keras.layers.Input((256, 256, 3))
model = create_unet(inputs)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = "Adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
tf.keras.utils.plot_model(model, show_shapes= True)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau,TensorBoard

early_stopping = EarlyStopping(monitor='val_acc', patience=3)
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=1e-4)
tensorboard = TensorBoard(log_dir='./logs')

model_history = model.fit(
    train_samples["images"], 
    train_samples["masks"], 
    validation_data = (test_samples["images"],test_samples["masks"]),
    callbacks = [early_stopping, checkpoint,reduce_lr, tensorboard],
    batch_size = 32, 
    epochs=200, 
    verbose = 1)

In [ ]:
plt.plot(model_history.history["accuracy"], label="training_accuracy")
plt.plot(model_history.history["val_accuracy"], label="val_accuracy")
plt.legend()
plt.grid(True)

In [ ]:
plt.plot(model_history.history["loss"], label="training_loss")
plt.plot(model_history.history["val_loss"], label="val_loss")
plt.legend()
plt.grid(True)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./logs

In [ ]:
def predict_test_sample(val_map, model):
  img = val_map["images"]
  masks = val_map["masks"]

  test_image  = np.array(img)
  predictions = model.predict(test_image)
  return predictions, test_image, masks

In [ ]:
def plot_images(test_image, predict, groun_truth):
  plt.figure(figsize=(20,20))

  plt.subplot(1, 3, 1)
  plt.imshow(test_image)
  plt.title("Image")

  plt.subplot(1,3,2)
  plt.imshow(predict)
  plt.title("Precited mask")

  plt.subplot(1,3,3)
  plt.imshow(groun_truth)
  plt.title("Ground truth mask")


In [ ]:
predicted_masks, test_images, ground_truth_mask = predict_test_sample(test_samples, model)

In [ ]:
plot_images(test_images[3], predicted_masks[3], ground_truth_mask[3])